In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
def find_eig(laplacian):
    eigenvalues, eigenvectors = np.linalg.eig(laplacian)
    sorted_indices = np.argsort(eigenvalues)
    eigenvalues, eigenvectors = eigenvalues[sorted_indices], eigenvectors[:, sorted_indices]
#     print(eigenvalues,'\n'*2 ,eigenvectors)
    return eigenvalues, eigenvectors

In [ ]:
def find_eig_torch(laplacian: torch.Tensor):
    eigenvalues, eigenvectors = torch.linalg.eig(laplacian)
    eigenvalues = eigenvalues.to(float)
    eigenvectors = eigenvectors.to(float)
    sorted_indices = torch.argsort(eigenvalues)
    eigenvalues, eigenvectors = eigenvalues[sorted_indices], eigenvectors[:, sorted_indices]
#     print(eigenvalues,'\n'*2 ,eigenvectors)
    return eigenvalues, eigenvectors

In [ ]:
def print_eig(eigenvalues, eigenvectors, n_comp = 5):
    eigen_sum = eigenvalues.sum()
    eigenvalues = eigenvalues[:n_comp]
    figsize = (len(eigenvalues)*3, 14)
    
    fig, ax = plt.subplots(len(eigenvalues), 2, figsize=figsize, sharex=True)
    for n in range(len(eigenvalues)):
        ax[n, 0].plot(eigenvectors[:,n], '*')
        ax[n, 0].set_title(f'{n}. eigenvalue = {eigenvalues[n]}')
    
    ax[0, 1].plot(eigenvalues, '*')
    ax[0, 1].set_title('Eigenvalues')
    plt.suptitle(f'Dirichlet energy: {eigen_sum}')

In [ ]:
def normalize_A_torch(a_m, d_m):
    return torch.sqrt(torch.linalg.inv(d_m))@a_m@torch.sqrt(torch.linalg.inv(d_m))

In [ ]:
def normalize_A(a_m, d_m):
    return np.sqrt(np.linalg.inv(d_m))@a_m@np.sqrt(np.linalg.inv(d_m))

#1

0, 1, 2, 3, 4

0 -> 1 -> 2


1 -> 3

3 -> 4

# Grafo reale

In [ ]:
## caso intermedio
# A = np.array([
#     [0, 1, 1, 0, 0], 
#     [1, 0, 1, 1, 0],
#     [1, 1, 0, 0, 0],
#     [0, 1, 0, 0, 1],
#     [0, 0, 0, 1, 0]
# ]).astype(float)

# caso con due grafi disconnessi
A = np.array([
    [0, 1, 1, 0, 0], 
    [1, 0, 1, 0, 0],
    [1, 1, 0, 0, 0],
    [0, 0, 0, 0, 1],
    [0, 0, 0, 1, 0]
]).astype(float)

# tutto connesso
# A = np.array([
#     [0, 1, 1, 1, 1], 
#     [1, 0, 1, 1, 1],
#     [1, 1, 0, 1, 1],
#     [1, 1, 1, 0, 1],
#     [1, 1, 1, 1, 0]
# ]).astype(float)

# # tutto sconnesso
# A = np.ones((100, 32))

# A += np.eye(A.shape[0])

In [ ]:
D = np.diag(A.sum(1))
L_norm = np.eye(A.shape[0])-normalize_A(A, D)
L = D-A

In [ ]:
L_norm = np.eye(A.shape[0])-normalize_A(A, D)

In [ ]:
L = D-A
L

In [ ]:
# eigenvalues, eigenvectors = find_eig(L)

# print_eig(eigenvalues, eigenvectors)
eigenvalues, eigenvectors = find_eig(L)

print_eig(eigenvalues, eigenvectors)


L'energia di dirichlet si azzera quando tutti i punti sono disconnessi tra loro

# Grafo da dati

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# temperature
sigma=1.

data = torch.Tensor([
    [1, 2, 1, 2],
    [1, 1, 1, 2],
    [3, 3, 3, 2],
    [18, 4, 4, 15],
    [19, 10, 10, 14],
]).to('cuda')

# data = torch.Tensor([
#     [1, 2, 1, 2],
#     [1, 1, 1, 2],
#     [1, 1, 1, 1],
#     [2, 1.2, 1, 2],
#     [1, 1.2, 2, 1.2],
# ]).to('cuda')

# data = np.array([
#     [5, 5, 5, 5],
#     [10, 10, 10, 10],
#     [1, 1, 1, 1],
#     [40, 40, 40, 40],
#     [500, 500, 500, 500],
# ])
# data = np.random.rand(3000,32)
data = torch.rand((4000, 32)).to('cuda')

In [ ]:
def calc_D_L_torch(data: torch.Tensor):
    dist_matrix = ((data[None, ...] - data.unsqueeze(1)) ** 2).sum(-1)
    # compute affinity matrix
    A = torch.exp(-dist_matrix / (sigma ** 2))

    # compute degree matrix
    D = torch.diag(A.sum(1))
    # compute laplacian
    L = D - A
    return A, D, L

In [ ]:
%%time

A, D, L = calc_D_L_torch(data)
L_norm = torch.eye(A.shape[0]).to('cuda')-normalize_A_torch(A, D)

eigenvalues, eigenvectors = find_eig_torch(L)

In [ ]:
print_eig(eigenvalues.cpu().numpy(), eigenvectors.cpu().numpy())

- energia del laplaciano normalizzato va da 0 al numero di sample utilizzati per la costruzione del grafo - 1
- energia del laplaciano va da 0 a n_sample*(n_sample-1)

In [ ]:
eigenvalues.numpy()